In [8]:
import os
import sys
sys.path.append(os.path.abspath('../gmid_repo/mosplot/src'))

try:
    from main import load_lookup_table, LoadMosfet
    print("Successfully imported from src.main!")
except ImportError as e:
    print(f"Error importing directly from src.main: {e}")

Successfully imported from src.main!


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interactive_output, HBox, VBox
import matplotlib.ticker as ticker 

In [10]:
nmos_lv_path = '../gmid_repo/LUTs/nmos_lv_lut_tt.npy'
pmos_lv_path = '../gmid_repo/LUTs/pmos_lv_lut_tt.npy'


lookup_table_nmos = load_lookup_table(nmos_lv_path)
lookup_table_pmos = load_lookup_table(pmos_lv_path)

In [11]:
nmos = LoadMosfet(lookup_table=lookup_table_nmos, mos="nmos", vsb=0.0, vds=0.6)
pmos = LoadMosfet(lookup_table=lookup_table_pmos, mos="pmos", vsb=0.0, vds=-0.6, vgs=(-1.2, -0.15))


In [12]:
def plot_data_vs_data(x_values, y_values, z_values, length, x_axis_name, y_axis_name='y', y_multiplier=1, log=False):
    x_values_flat = np.array(x_values).flatten()
    y_values_flat = np.array(y_values, dtype=np.float64).flatten()
    z_values_flat = np.array(z_values, dtype=np.float64).flatten()
    length_flat = np.array(length).flatten()

    # Ensure all inputs have the same length
    if not (len(x_values_flat) == len(y_values_flat) == len(z_values_flat) == len(length_flat)):
        raise ValueError("All input arrays (x_values, y_values, z_values, length) must have the same number of elements.")

    unique_lengths = np.unique(length_flat)
    unique_lengths_in_micro = unique_lengths * 1e6

    def update_plot(selected_length, x_value=None, y_value=None):
        plt.figure(figsize=(12, 8))  # Make the figure wider (adjust as needed)

        if selected_length == "Show All":
            mask = np.ones_like(length_flat, dtype=bool)
        else:
            selected_length_in_micro = float(selected_length.replace(' μm', ''))
            tolerance = 0.01  # Tighten the tolerance to avoid unwanted data points
            mask = np.abs(length_flat * 1e6 - selected_length_in_micro) < tolerance

        # Apply the mask to the data
        x_values_for_length = x_values_flat[mask]
        y_values_for_length = y_values_flat[mask] * y_multiplier
        z_values_for_length = z_values_flat[mask]
        length_for_length = length_flat[mask] * 1e6

        if selected_length == "Show All":
            for length_value in np.unique(length_for_length):
                mask_all = (length_for_length == length_value)
                plt.plot(x_values_for_length[mask_all], y_values_for_length[mask_all])

            min_length = np.min(unique_lengths_in_micro)
            max_length = np.max(unique_lengths_in_micro)
            plt.title(f'{y_axis_name} vs {x_axis_name} (Length from {min_length:.2f} μm to {max_length:.2f} μm)')

        else:
            plt.plot(x_values_for_length, y_values_for_length)
            plt.title(f'{y_axis_name} vs {x_axis_name} for {selected_length}')

        plt.xlabel(f'{x_axis_name}')
        plt.ylabel(f'{y_axis_name}')

        if log:
            plt.yscale('log')
            plt.gca().yaxis.set_major_locator(ticker.LogLocator(base=10, subs=[], numticks=10))
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'$10^{int(np.log10(x))}$'))
            plt.ylabel(f'{y_axis_name} (Log Base 10)')

        if y_value is not None and x_value_widget.disabled:
            closest_index = np.abs(y_values_for_length - y_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            corresponding_z = z_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='blue', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            z_value_widget.value = corresponding_z
            print(f"The corresponding {x_axis_name} value for {y_axis_name} = {closest_y:.2f} is: {closest_x:.2f}")
        elif x_value is not None and y_value_widget.disabled:
            closest_index = np.abs(x_values_for_length - x_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            corresponding_z = z_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='red', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            z_value_widget.value = corresponding_z
            print(f"The corresponding {y_axis_name} value for {x_axis_name} = {closest_x:.2f} is: {closest_y:.2f}")

        plt.grid(True)
        plt.legend()
        plt.show()

    dropdown_options = ["Show All"] + [f'{length:.2f} μm' for length in unique_lengths_in_micro]
    length_widget = widgets.Dropdown(
        options=dropdown_options,
        value=dropdown_options[0],
        description='Length:',
        layout=widgets.Layout(width='500px')  # Make the dropdown wider
    )

    x_value_widget = widgets.FloatText(
        value=np.mean(x_values_flat),
        description=f"{x_axis_name}:",
        disabled=False,
        layout=widgets.Layout(width='300px', margin='0 40px 0 0'),  # Push input boxes more to the right
        description_width='150px'  # Smaller description width
    )

    y_value_widget = widgets.FloatText(
        value=None,
        description=f"{y_axis_name}:",
        disabled=True,
        layout=widgets.Layout(width='300px', margin='0 40px 0 0'),  # Push input boxes more to the right
        description_width='150px'  # Smaller description width
    )

    z_value_widget = widgets.FloatText(
        value=None,
        description=f" Vgs:",
        disabled=True,
        layout=widgets.Layout(width='300px', margin='0 40px 0 0'),  # Push input boxes more to the right
        description_width='150px'  # Smaller description width
    )

    select_x_or_y_widget = widgets.Checkbox(
        value=True,
        description=f"{x_axis_name} (uncheck for {y_axis_name})",
        layout=widgets.Layout(width='300px')  # Make the checkbox wider
    )

    def toggle_x_or_y(change):
        if change['new']:
            x_value_widget.disabled = False
            y_value_widget.disabled = True
        else:
            x_value_widget.disabled = True
            y_value_widget.disabled = False

    select_x_or_y_widget.observe(toggle_x_or_y, names='value')

    output = interactive_output(update_plot, {
        'selected_length': length_widget,
        'x_value': x_value_widget,
        'y_value': y_value_widget
    })

    display(VBox([length_widget, select_x_or_y_widget, HBox([x_value_widget, y_value_widget]), z_value_widget, output]))
    
def display_resistance(ro_value):
    """Determine the resistance value and its unit."""
    if ro_value < 1e3:
        return ro_value, "Ω"
    elif ro_value < 1e6:
        return ro_value / 1e3, "kΩ"
    elif ro_value < 1e9:
        return ro_value / 1e6, "MΩ"
    else:
        return ro_value / 1e9, "GΩ"

def display_current(Id_value):
    """Determine the current value and its unit."""
    if Id_value < 1e-6:
        return Id_value * 1e9, "nA"  # Convert to nA
    elif Id_value < 1e-3:
        return Id_value * 1e6, "μA"  # Convert to μA
    else:
        return Id_value * 1e3, "mA"   # Convert to mA
    
def dB_to_linear(av_db):
    return 10 ** (av_db / 20)


def determine_inversion_region(gm_id_value, device_type):
    """Determine the inversion region based on gm/id value for NMOS or PMOS."""
    if device_type == 'nmos':
        if gm_id_value > 20:
            return "Weak Inversion"
        elif 10 < gm_id_value <= 20:
            return "Moderate Inversion"
        else:
            return "Strong Inversion"
    elif device_type == 'pmos':
        if gm_id_value > 20:
            return "Weak Inversion"
        elif 10 < gm_id_value <= 20:
            return "Moderate Inversion"
        else:
            return "Strong Inversion"
    else:
        raise ValueError("Invalid device type. Use 'nmos' or 'pmos'.")
    


# NMOS GMID

In [13]:
width_values = nmos.extracted_table['width']
id_values = nmos.extracted_table['id']
gm_values = nmos.extracted_table['gm']
gds_values = nmos.extracted_table['gds']
vgs_values= nmos.extracted_table['vgs']

plot_data_vs_data(gm_values/id_values, gm_values/gds_values, vgs_values, nmos.extracted_table['lengths'], 'gm/id', 'gm/gds')

# PMOS GMID

In [14]:
width_values = pmos.extracted_table['width']
id_values = pmos.extracted_table['id']
gm_values = pmos.extracted_table['gm']
gds_values = pmos.extracted_table['gds']
vgs_values= pmos.extracted_table['vgs']

plot_data_vs_data(gm_values/id_values, gm_values/gds_values, vgs_values, pmos.extracted_table['lengths'], 'gm/id', 'gm/gds')